In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from PIL import Image
from io import StringIO,BytesIO 
from scipy.spatial.distance import pdist
import cv2
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc 
from functools import reduce
import wfdb#https://github.com/MIT-LCP/wfdb-python
from wfdb import processing
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.set_device(1)
print (torch.cuda.current_device())

Loading faiss with AVX2 support.


1


In [2]:
#Read data with List storage Name:[name],I:[img],Y[type]
def TypetoNum(itype): #map the type into number.
    if itype =='AMD': return 0
    elif itype =='DR': return 1
    elif itype =='glaucoma': return 2
    elif itype =='myopia': return 3
    else: return 4 #norm
    
root_dir = '/data/fjsdata/fundus/iSee/iSee_multi_dataset/' #the path of images
trainset = pd.read_csv("/data/fjsdata/fundus/iSee/iSee_multi_dataset/CBIR_iSee_train.csv" , sep=',')#load trainset
testset = pd.read_csv("/data/fjsdata/fundus/iSee/iSee_multi_dataset/CBIR_iSee_test.csv" , sep=',')#load testset
tstart = time.time()
#read train image with CV
trN, trI, trY = [],[],[]
norm = 6993
for iname, itype in np.array(trainset).tolist():
    if iname.endswith(".jpg"):
        try:
            image_dir = root_dir+'img_data_%s'%itype
            image_path = os.path.join(image_dir, iname)
            if itype == 'norm':
                if norm>0:
                    img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(1920,1920,3)->(256,256,3)
                    trN.append(iname)
                    trI.append(img)
                    trY.append(TypetoNum(itype))
                    norm = norm - 1
            else:
                img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(1920,1920,3)->(256,256,3)
                trN.append(iname)
                trI.append(img)
                trY.append(TypetoNum(itype))    
        except:
            print(iname+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(trN),trainset.shape[0]))
        sys.stdout.flush()
print('The length of train set is %d'%len(trN))
#read test image with CV
teN, teI, teY = [],[],[]
norm = 777
for iname, itype in np.array(testset).tolist():
    if iname.endswith(".jpg"):
        try:
            image_dir = root_dir+'img_data_%s'%itype
            image_path = os.path.join(image_dir, iname)
            if itype == 'norm':
                if norm>0:
                    img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(1920,1920,3)->(256,256,3)
                    teN.append(iname)
                    teI.append(img)
                    teY.append(TypetoNum(itype))
                    norm = norm - 1
            else:
                img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(1920,1920,3)->(256,256,3)
                teN.append(iname)
                teI.append(img)
                teY.append(TypetoNum(itype)) 
        except:
            print(iname+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(teN),testset.shape[0]))
        sys.stdout.flush()
print('The length of test set is %d'%len(teN))
elapsed = time.time() - tstart    
print('Completed data handle in %d seconds' % int(elapsed))

9000 / 9000 The length of train set is 9000
1000 / 1000 The length of test set is 1000
Completed data handle in 770 seconds


In [3]:
class SpatialAttention(nn.Module):#spatial attention layer
    def __init__(self):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size=3, padding=1, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
    
class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels, out_channels=out_channels,
                kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

        self.downsample_layer = None
        self.do_downsample = False
        if in_channels != out_channels or stride != 1:
            self.do_downsample = True
            self.downsample_layer = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        # initialize weights
        self.apply(self.init_weights)

    def forward(self, x):
        identity = x
        out = self.net(x)

        if self.do_downsample:
            identity = self.downsample_layer(x)

        return F.relu(out + identity, inplace=True) #resnet

    @staticmethod
    def init_weights(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            
class ATHNet(nn.Module):
    def __init__(self, hash_size: int, type_size: int):
        super().__init__()
        #resnet and maxpool
        self.net1 = nn.Sequential(#(3,256,256)->(16,128,128)
            ResBlock(in_channels=3, out_channels=16, stride=2), 
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1)
        )
        
        #Attention (16,128,128)->(16,128,128)
        self.sa = SpatialAttention()
        
        #resnet and meanpool
        self.net2 =nn.Sequential( #(16,128,128)->(8,64,64)
            ResBlock(in_channels=16, out_channels=8, stride=2),
            nn.AvgPool2d(kernel_size=3, padding=1, stride=1)
        ) 
         
        #fully connected with conv (8,64,64)->(1,32,32)
        self.dense=ResBlock(in_channels=8, out_channels=1, stride=2)
        #fully connected (1,32,32)->class_size
        self.hashlayer = nn.Linear(1*32*32, hash_size)
        self.typelayer = nn.Linear(1*32*32, type_size)
    
        # initialize weights
        self.apply(self.init_weights)

    def forward(self, x):
        x = self.net1(x)
        x = self.sa(x)*x
        x = self.net2(x)
        x = self.dense(x)
        x = x.view(x.size(0),-1)
        x_hash = self.hashlayer(x)
        x_type = self.typelayer(x)
        return x_hash, x_type

    @staticmethod
    def init_weights(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)

class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
        self.mse_loss = nn.MSELoss(reduction='none')
        self.l1_loss = nn.L1Loss(reduction='mean')
    
    def forward(self,h1,h2,y):    
        margin_val = self.margin * h1.shape[1]
        squared_loss = torch.mean(self.mse_loss(h1, h2), dim=1)
        # T1: 0.5 * (1 - y) * dist(x1, x2)
        positive_pair_loss = (0.5 * (1 - y) * squared_loss)
        mean_positive_pair_loss = torch.mean(positive_pair_loss)
        # T2: 0.5 * y * max(margin - dist(x1, x2), 0)
        zeros = torch.zeros_like(squared_loss)
        marginMat = margin_val * torch.ones_like(squared_loss)
        negative_pair_loss = 0.5 * y * torch.max(zeros, marginMat - squared_loss)
        mean_negative_pair_loss = torch.mean(negative_pair_loss)

        # T3: alpha(dst_l1(abs(x1), 1)) + dist_l1(abs(x2), 1)))
        mean_value_regularization = self.alpha * (
                self.l1_loss(torch.abs(h1), torch.ones_like(h1)) +
                self.l1_loss(torch.abs(h2), torch.ones_like(h2)))

        loss = mean_positive_pair_loss + mean_negative_pair_loss + mean_value_regularization
        return loss
    
#https://github.com/luyajie/triplet-deep-hash-pytorch#triplet-deep-hash-pytorch            
class TripletLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(TripletLoss, self).__init__()
        self.margin = margin #margin threshold
        self.mse_loss = nn.MSELoss(reduction='none')
    
    def forward(self,H_q,H_p,H_n):    
        margin_val = self.margin * H_q.shape[1]
        squared_loss_pos = torch.mean(self.mse_loss(H_q, H_p), dim=1)
        squared_loss_neg = torch.mean(self.mse_loss(H_q, H_n), dim=1)
        zeros = torch.zeros_like(squared_loss_neg)
        loss  = torch.max(zeros, margin_val - squared_loss_neg + squared_loss_pos)
        return torch.mean(loss)

class FocalLoss(nn.Module):
    #Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, out, y):
        y = y.view(-1,1)
        logpt = F.log_softmax(out,dim=1)#default ,dim=1
        logpt = logpt.gather(1,y)# dim=1, index=y, max
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=out.data.type():
                self.alpha = self.alpha.type_as(out.data)
            at = self.alpha.gather(0,y.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

class CircleLoss(nn.Module):
    def __init__(self, scale=32, margin=0.25, similarity='cos', **kwargs):
        super(CircleLoss, self).__init__()
        self.scale = scale
        self.margin = margin
        self.similarity = similarity

    def forward(self, feats, labels):
        assert feats.size(0) == labels.size(0), \
            f"feats.size(0): {feats.size(0)} is not equal to labels.size(0): {labels.size(0)}"
        batch_size = feats.size(0)
        if self.similarity == 'dot':
            sim_mat = torch.matmul(feats, torch.t(feats))
        elif self.similarity == 'cos':
            feats = F.normalize(feats)
            sim_mat = feats.mm(feats.t())
        else:
            raise ValueError('This similarity is not implemented.')
        loss = list()
        for i in range(batch_size):
            pos_index = labels == labels[i]
            pos_index[i] = 0
            neg_index = labels != labels[i]
            pos_pair_ = sim_mat[i][pos_index]
            neg_pair_ = sim_mat[i][neg_index]

            alpha_p = torch.relu(-pos_pair_ + 1 + self.margin)
            alpha_n = torch.relu(neg_pair_ + self.margin)
            margin_p = 1 - self.margin
            margin_n = self.margin
            loss_p = torch.sum(torch.exp(-self.scale * alpha_p * (pos_pair_ - margin_p)))
            loss_n = torch.sum(torch.exp(self.scale * alpha_n * (neg_pair_ - margin_n)))
            loss.append(torch.log(1 + loss_p * loss_n))

        loss = sum(loss) / batch_size
        return loss

#Generate image pairs for model
def onlineGenImgPairs():
    if (len(trY) % 2) == 0: spls = len(trY)
    else:  spls = len(trY)-1
    idx_sf = random.sample(range(0, spls),spls)
    trI1_sf, trI2_sf, trY1_sf, trY2_sf = [],[],[],[]
    flag = 0
    for i in idx_sf:
        if flag==0:
            trI1_sf.append(trI[i])
            trY1_sf.append(trY[i])
            flag =1
        else:
            trI2_sf.append(trI[i])
            trY2_sf.append(trY[i])
            flag =0
    trY_sf = np.where((np.array(trY1_sf)-np.array(trY2_sf))!=0,1,0)
    return np.array(trI1_sf),np.array(trI2_sf),trY_sf

#Generate image pairs for model
def onlineGenImgTriplets( ):
    idx_sf = []
    idx_0 = np.where( np.array(trY) == 0 ) #class 0
    idx_0 = list(idx_0[0])#[0:4555]
    idx_sf.extend(idx_0)
    idx_1 = np.where( np.array(trY) == 1 ) #class 1
    idx_1 = list(idx_1[0])
    idx_sf.extend(idx_1)
    idx_2 = np.where( np.array(trY) == 2 ) #class 2
    idx_2 = list(idx_2[0])
    idx_sf.extend(idx_2)
    idx_3 = np.where( np.array(trY) == 3 ) #class 3
    idx_3 = list(idx_3[0])
    idx_sf.extend(idx_3)
    idx_4 = np.where( np.array(trY) == 4 ) #class 4
    idx_4 = list(idx_4[0])
    idx_sf.extend(idx_4)
    random.shuffle(idx_sf)   
    trQ_sf, trP_sf, trN_sf = [], [], []
    trQ_y, trP_y, trN_y = [], [], []
    for iQ in idx_sf:
        trQ_sf.append(trI[iQ])
        trQ_y.append(trY[iQ])
        if trY[iQ] == 0:
            idx_tmp = idx_0.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_0))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 1:
            idx_tmp = idx_1.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_1))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 2:
            idx_tmp = idx_2.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_2))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 3:
            idx_tmp = idx_3.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_3))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 4:
            idx_tmp = idx_4.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_4))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        else: pass
        sys.stdout.write('\r{} / {} '.format(len(trQ_sf),len(idx_sf)))
        sys.stdout.flush()
    return np.array(trQ_sf),np.array(trP_sf),np.array(trN_sf), np.array(trQ_y), np.array(trP_y), np.array(trN_y)

In [11]:
#trQ_sf, trP_sf, trN_sf, trQ_y, trP_y, trN_y = onlineGenImgTriplets() #sample  triplet labels
assert (trQ_sf.shape==trP_sf.shape and trQ_sf.shape==trN_sf.shape)
assert (trQ_y.shape==trP_y.shape and trQ_y.shape==trN_y.shape)
assert (np.mean(np.where((np.array(trQ_y)-np.array(trP_y))!=0,1,0))==0.0)
assert (np.mean(np.where((np.array(trQ_y)-np.array(trN_y))!=0,1,0))==1.0)

#define model
model = ATHNet(hash_size=36, type_size=5).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
tl_loss  = TripletLoss(margin=0.5).cuda() #define TripletLoss 
ce_loss  = nn.CrossEntropyLoss().cuda() #define ce mutli-classes

#train model
best_net, best_loss = None, float('inf')
batchSize = 10
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trQ_sf)))
    train_q = trQ_sf[shuffled_idx]
    train_q_y = trQ_y[shuffled_idx]
    train_p = trP_sf[shuffled_idx]
    train_p_y = trP_y[shuffled_idx]
    train_n = trN_sf[shuffled_idx]
    train_n_y = trN_y[shuffled_idx]
    num_batches = len(trQ_sf) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trQ_sf), (i+1)*batchSize])
        Q_batch = torch.from_numpy(train_q[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Q_y_batch = torch.from_numpy(train_q_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        P_batch = torch.from_numpy(train_p[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        P_y_batch = torch.from_numpy(train_p_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        N_batch = torch.from_numpy(train_n[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        N_y_batch = torch.from_numpy(train_n_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        Q_hash, Q_type = model(Q_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        P_hash, P_type = model(P_batch.permute(0, 3, 1, 2))
        N_hash, N_type = model(N_batch.permute(0, 3, 1, 2))
        #loss
        hash_loss = tl_loss(Q_hash,P_hash,N_hash)
        type_loss = ce_loss(Q_type,Q_y_batch) + ce_loss(P_type,P_y_batch) + ce_loss(N_type,N_y_batch) #F.log_softmax+F.nll_loss
        loss = hash_loss+type_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
#release gpu memory
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()

#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teY_pred = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    x_hash, x_type = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(x_hash.cpu().data.numpy().tolist()) #record feature
    x_type = F.log_softmax(x_type,dim=1) 
    pred = x_type.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

#performance of retrieval
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):
        stype = teY[i]
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance of classification
print ( 'Accuracy: %.6f'%accuracy_score(teY, teY_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, teY_pred, labels=labels )
print (cm)
print ('Sensitivity of AMD: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of DR: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of glaucoma: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of myopia: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of normal: %.6f'%float(cm[4][4]/np.sum(cm[4])))

 900 / 900 : loss = 22.319298Eopch:     1 mean_loss = 20.938066
 900 / 900 : loss = 16.338124Eopch:     2 mean_loss = 19.773190
 900 / 900 : loss = 12.743713Eopch:     3 mean_loss = 18.801887
 900 / 900 : loss = 9.6762932Eopch:     4 mean_loss = 18.013169
 900 / 900 : loss = 20.730305Eopch:     5 mean_loss = 16.205706
 900 / 900 : loss = 16.390224Eopch:     6 mean_loss = 16.055036
 900 / 900 : loss = 24.501198Eopch:     7 mean_loss = 15.166719
 900 / 900 : loss = 14.856543Eopch:     8 mean_loss = 14.857771
 900 / 900 : loss = 18.3908263Eopch:     9 mean_loss = 14.786935
 900 / 900 : loss = 8.6421152Eopch:    10 mean_loss = 14.408413
best_loss = 14.408413
 99 / 100  Completed buliding index in 1 seconds
topk=5, mHR=0.741800, mAP=0.681353, mRR=0.892617
topk=10, mHR=0.744800, mAP=0.668303, mRR=0.877998
topk=15, mHR=0.744267, mAP=0.661416, mRR=0.860988
topk=20, mHR=0.743450, mAP=0.656055, mRR=0.850868
Accuracy: 0.780000
[[  2   0   0  12  58]
 [  0   0   0   3  24]
 [  0   0   0   4  41]
 

In [12]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.108333, mAP=0.042545, mRR=1.000000
label=1, mHR=0.018519, mAP=0.008560, mRR=1.000000
label=2, mHR=0.051111, mAP=0.019489, mRR=0.500000
label=3, mHR=0.856962, mAP=0.799538, mRR=0.250000
label=4, mHR=0.857786, mAP=0.773447, mRR=1.000000


In [17]:
#define model
model = ATHNet(hash_size=36, type_size=5).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
tl_loss  = TripletLoss(margin=0.5).cuda() #define TripletLoss 

#train model
best_net, best_loss = None, float('inf')
batchSize = 10
pl_t_i = []
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trQ_sf)))
    train_q = trQ_sf[shuffled_idx]
    train_p = trP_sf[shuffled_idx]
    train_n = trN_sf[shuffled_idx]
    num_batches = len(trQ_sf) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trQ_sf), (i+1)*batchSize])
        Q_batch = torch.from_numpy(train_q[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        P_batch = torch.from_numpy(train_p[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        N_batch = torch.from_numpy(train_n[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        Q_hash, _ = model(Q_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        P_hash, _ = model(P_batch.permute(0, 3, 1, 2))
        N_hash, _ = model(N_batch.permute(0, 3, 1, 2))
        #loss
        loss = tl_loss(Q_hash,P_hash,N_hash)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    pl_t_i.append(np.mean(losses))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
#release gpu memory
model = model.cpu()
tl_loss=tl_loss.cpu()
torch.cuda.empty_cache()

#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize 
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teY_pred = []
teY_prob = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    x_hash, x_type = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(x_hash.cpu().data.numpy().tolist()) #record feature
    x_type = F.log_softmax(x_type,dim=1) 
    teY_prob.extend(x_type.cpu().data.numpy().tolist())
    pred = x_type.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#performance of retrieval
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):
        stype = teY[i]
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance of classification
print ( 'Accuracy: %.6f'%accuracy_score(teY, teY_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, teY_pred, labels=labels )
print (cm)
print ('Sensitivity of AMD: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of DR: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of glaucoma: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of myopia: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of normal: %.6f'%float(cm[4][4]/np.sum(cm[4])))

 900 / 900 : loss = 16.148705Eopch:     1 mean_loss = 17.281924
 900 / 900 : loss = 14.832659Eopch:     2 mean_loss = 16.533100
 900 / 900 : loss = 12.735866Eopch:     3 mean_loss = 15.731899
 900 / 900 : loss = 13.043617Eopch:     4 mean_loss = 14.589371
 900 / 900 : loss = 9.9067984Eopch:     5 mean_loss = 13.887730
 900 / 900 : loss = 9.7077581Eopch:     6 mean_loss = 12.143182
 900 / 900 : loss = 1.9319865Eopch:     7 mean_loss = 11.018245
 900 / 900 : loss = 0.0032887Eopch:     8 mean_loss = 9.473015
 900 / 900 : loss = 1.8839068Eopch:     9 mean_loss = 8.116827
 900 / 900 : loss = 10.913944Eopch:    10 mean_loss = 7.051661
best_loss = 7.051661
 99 / 100  Completed buliding index in 1 seconds
topk=5, mHR=0.752400, mAP=0.712127, mRR=0.929647
topk=10, mHR=0.747200, mAP=0.693772, mRR=0.907191
topk=15, mHR=0.747467, mAP=0.686191, mRR=0.885441
topk=20, mHR=0.744950, mAP=0.680233, mRR=0.876510
Accuracy: 0.087000
[[ 20   0  15  35   2]
 [ 13   0   3   9   2]
 [ 14   0   5  23   3]
 [ 37 

In [18]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.050000, mAP=0.016845, mRR=0.166667
label=1, mHR=0.011111, mAP=0.002844, mRR=0.500000
label=2, mHR=0.020000, mAP=0.006206, mRR=0.333333
label=3, mHR=0.445570, mAP=0.338441, mRR=0.333333
label=4, mHR=0.910167, mAP=0.856456, mRR=1.000000


In [21]:
#define model
model = ATHNet(hash_size=36, type_size=5).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = HashLossFunc(margin=0.5).cuda() #define PairwiseLoss 
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
for epoch in range(10):#iteration
    trI1_sf, trI2_sf, trY_sf = onlineGenImgPairs()
    losses = []
    num_batches = len(trY_sf) // batchSize 
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY_sf), (i+1)*batchSize])
        I1_batch = torch.from_numpy(trI1_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        I2_batch = torch.from_numpy(trI2_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(trY_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        X1_batch,_ = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        X2_batch,_ = model(I2_batch.permute(0, 3, 1, 2))
        #binary-like loss
        loss = criterion(X1_batch,X2_batch,Y_batch)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()
#hash code of train data from model
#torch.cuda.synchronize()
batchSize = 10
num_batches = len(trI) // batchSize 
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
#torch.cuda.synchronize()
teY_pred = []
teY_prob = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    x_hash, x_type = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(x_hash.cpu().data.numpy().tolist()) #record feature
    x_type = F.log_softmax(x_type,dim=1) 
    teY_prob.extend(x_type.cpu().data.numpy().tolist())
    pred = x_type.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#performance of retrieval
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):
        stype = teY[i]
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance of classification
print ( 'Accuracy: %.6f'%accuracy_score(teY, teY_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, teY_pred, labels=labels )
print (cm)
print ('Sensitivity of AMD: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of DR: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of glaucoma: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of myopia: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of normal: %.6f'%float(cm[4][4]/np.sum(cm[4])))

 450 / 450 : loss = 4.500243Eopch:     1 mean_loss = 3.455760
 450 / 450 : loss = 1.800289Eopch:     2 mean_loss = 3.436414
 450 / 450 : loss = 3.600247Eopch:     3 mean_loss = 3.508299
 450 / 450 : loss = 4.500104Eopch:     4 mean_loss = 3.438305
 450 / 450 : loss = 2.700234Eopch:     5 mean_loss = 3.460263
 450 / 450 : loss = 3.600263Eopch:     6 mean_loss = 3.418257
 450 / 450 : loss = 2.700142Eopch:     7 mean_loss = 3.444215
 450 / 450 : loss = 4.500175Eopch:     8 mean_loss = 3.466228
 450 / 450 : loss = 2.700137Eopch:     9 mean_loss = 3.462227
 450 / 450 : loss = 5.400177Eopch:    10 mean_loss = 3.456213
best_loss = 3.418257
 99 / 100  Completed buliding index in 1 seconds
topk=5, mHR=0.641200, mAP=0.568637, mRR=0.866067
topk=10, mHR=0.638900, mAP=0.545127, mRR=0.836613
topk=15, mHR=0.636267, mAP=0.532407, mRR=0.807231
topk=20, mHR=0.635300, mAP=0.526204, mRR=0.798198
Accuracy: 0.046000
[[  0   0  72   0   0]
 [  0   0  27   0   0]
 [  0   0  45   0   0]
 [  0   0  79   0   0]


In [22]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.091667, mAP=0.032857, mRR=0.111111
label=1, mHR=0.033333, mAP=0.008920, mRR=0.166667
label=2, mHR=0.031111, mAP=0.012204, mRR=0.166667
label=3, mHR=0.073418, mAP=0.037558, mRR=0.100000
label=4, mHR=0.803990, mAP=0.693506, mRR=1.000000


In [26]:
#define model
model = ATHNet(hash_size=36, type_size=5).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = CircleLoss().cuda() #define TripletLoss 
#train model
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
trI = np.array(trI)
trY = np.array(trY)
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trY)))
    train_x = trI[shuffled_idx]
    train_y = trY[shuffled_idx]
    num_batches = len(trY) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY), (i+1)*batchSize])
        x_batch = torch.from_numpy(train_x[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        y_batch = torch.from_numpy(train_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        out_batch,_ = model(x_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        loss = criterion(out_batch,y_batch) #F.log_softmax+F.nll_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
#release gpu memory
model = model.cpu()
criterion=criterion.cpu()
torch.cuda.empty_cache()

#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
#torch.cuda.synchronize()
teF = []
num_batches = len(teI) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):#perfromance
        stype = teY[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance
scores, neighbors = gpu_index.search(np.ascontiguousarray(teF, dtype=np.float32), k=1) #return top1
y_pred = []
for i in neighbors.flatten():
    y_pred.append(np.array(trY)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(teY, y_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, y_pred, labels=labels )
print (cm)
print ('Sensitivity of AMD: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of DR: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of glaucoma: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of myopia: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of normal: %.6f'%float(cm[4][4]/np.sum(cm[4])))

 900 / 900 : loss = 17.224607Eopch:     1 mean_loss = 15.831477
 900 / 900 : loss = 15.681116Eopch:     2 mean_loss = 14.793125
 900 / 900 : loss = 13.446039Eopch:     3 mean_loss = 14.740195
 900 / 900 : loss = 14.352495Eopch:     4 mean_loss = 14.324153
 900 / 900 : loss = 14.953743Eopch:     5 mean_loss = 14.390648
 900 / 900 : loss = 12.884517Eopch:     6 mean_loss = 14.421440
 900 / 900 : loss = 0.0160846Eopch:     7 mean_loss = 14.171689
 900 / 900 : loss = 15.290313Eopch:     8 mean_loss = 14.023532
 900 / 900 : loss = 19.485395Eopch:     9 mean_loss = 13.866826
 900 / 900 : loss = 0.0528879Eopch:    10 mean_loss = 13.865439
best_loss = 13.865439
 99 / 100  Completed buliding index in 1 seconds
topk=5, mHR=0.659000, mAP=0.589640, mRR=0.869536
topk=10, mHR=0.660200, mAP=0.572216, mRR=0.835445
topk=15, mHR=0.659333, mAP=0.562605, mRR=0.814716
topk=20, mHR=0.660450, mAP=0.558988, mRR=0.800071
Accuracy: 0.654000
[[  3   5   3   3  58]
 [  3   1   2   1  20]
 [  2   0   2   4  37]
 [

In [27]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.072222, mAP=0.026461, mRR=0.100000
label=1, mHR=0.022222, mAP=0.007648, mRR=0.111111
label=2, mHR=0.051111, mAP=0.013616, mRR=0.142857
label=3, mHR=0.106329, mAP=0.049175, mRR=0.142857
label=4, mHR=0.828443, mAP=0.727936, mRR=1.000000


In [30]:
#define model
model = ATHNet(hash_size=36, type_size=5).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = FocalLoss(gamma=2,alpha=[0.2,0.4,0.2,0.15,0.05]).cuda() #define ce mutli-classes, 
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
trI = np.array(trI)
trY = np.array(trY)
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trY)))
    train_x = trI[shuffled_idx]
    train_y = trY[shuffled_idx]
    num_batches = len(trY) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY), (i+1)*batchSize])
        x_batch = torch.from_numpy(train_x[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        y_batch = torch.from_numpy(train_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        _, out_batch = model(x_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        loss = criterion(out_batch,y_batch) #F.log_softmax+F.nll_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
criterion = criterion.cpu()
torch.cuda.empty_cache()
#torch.cuda.synchronize()
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    feature, _ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    feature = feature.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(feature.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

teY_pred = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    feature, out_batch = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(feature.cpu().data.numpy().tolist()) #record feature
    out_batch = F.log_softmax(out_batch,dim=1) 
    pred = out_batch.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):#perfromance
        stype = teY[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance
scores, neighbors = gpu_index.search(np.ascontiguousarray(teF, dtype=np.float32), k=1) #return top1
y_pred = []
for i in neighbors.flatten():
    y_pred.append(np.array(trY)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(teY, y_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, y_pred, labels=labels )
print (cm)
print ('Sensitivity of AMD: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of DR: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of glaucoma: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of myopia: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of normal: %.6f'%float(cm[4][4]/np.sum(cm[4])))

 900 / 900 : loss = 0.022909Eopch:     1 mean_loss = 0.074986
 900 / 900 : loss = 0.027587Eopch:     2 mean_loss = 0.070604
 900 / 900 : loss = 0.025329Eopch:     3 mean_loss = 0.068125
 900 / 900 : loss = 0.012759Eopch:     4 mean_loss = 0.065778
 900 / 900 : loss = 0.091035Eopch:     5 mean_loss = 0.064244
 900 / 900 : loss = 0.047397Eopch:     6 mean_loss = 0.062789
 900 / 900 : loss = 0.050956Eopch:     7 mean_loss = 0.060335
 900 / 900 : loss = 0.024229Eopch:     8 mean_loss = 0.058508
 900 / 900 : loss = 0.018478Eopch:     9 mean_loss = 0.057175
 900 / 900 : loss = 0.074769Eopch:    10 mean_loss = 0.054883
best_loss = 0.054883
 99 / 100  Completed buliding index in 1 seconds
topk=5, mHR=0.639400, mAP=0.561873, mRR=0.861532
topk=10, mHR=0.640200, mAP=0.540552, mRR=0.832008
topk=15, mHR=0.637267, mAP=0.527894, mRR=0.808285
topk=20, mHR=0.632800, mAP=0.517455, mRR=0.789198
Accuracy: 0.648000
[[  5   2   6   4  55]
 [  4   0   4   2  17]
 [  3   6   4   5  27]
 [ 12   2   3   8  54]


In [31]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.070833, mAP=0.024496, mRR=0.333333
label=1, mHR=0.025926, mAP=0.007346, mRR=0.333333
label=2, mHR=0.066667, mAP=0.027205, mRR=0.200000
label=3, mHR=0.125316, mAP=0.049497, mRR=0.500000
label=4, mHR=0.799871, mAP=0.686557, mRR=1.000000


In [34]:
#define model
model = ATHNet(hash_size=36, type_size=5).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = nn.CrossEntropyLoss().cuda() #define ce mutli-classes
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
trI = np.array(trI)
trY = np.array(trY)
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trY)))
    train_x = trI[shuffled_idx]
    train_y = trY[shuffled_idx]
    num_batches = len(trY) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY), (i+1)*batchSize])
        x_batch = torch.from_numpy(train_x[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        y_batch = torch.from_numpy(train_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        _,out_batch = model(x_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        loss = criterion(out_batch,y_batch) #F.log_softmax+F.nll_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
criterion = criterion.cpu()
torch.cuda.empty_cache()
#torch.cuda.synchronize()
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    feature, _ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    feature = feature.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(feature.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

teY_pred = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    feature, out_batch = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(feature.cpu().data.numpy().tolist()) #record feature
    out_batch = F.log_softmax(out_batch,dim=1) 
    pred = out_batch.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):#perfromance
        stype = teY[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance
scores, neighbors = gpu_index.search(np.ascontiguousarray(teF, dtype=np.float32), k=1) #return top1
y_pred = []
for i in neighbors.flatten():
    y_pred.append(np.array(trY)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(teY, y_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, y_pred, labels=labels )
print (cm)
print ('Sensitivity of AMD: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of DR: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of glaucoma: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of myopia: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of normal: %.6f'%float(cm[4][4]/np.sum(cm[4])))

 900 / 900 : loss = 0.854811Eopch:     1 mean_loss = 0.842690
 900 / 900 : loss = 0.686369Eopch:     2 mean_loss = 0.815425
 900 / 900 : loss = 0.711693Eopch:     3 mean_loss = 0.800479
 900 / 900 : loss = 1.629473Eopch:     4 mean_loss = 0.781990
 900 / 900 : loss = 0.473299Eopch:     5 mean_loss = 0.763922
 900 / 900 : loss = 1.493669Eopch:     6 mean_loss = 0.748292
 900 / 900 : loss = 0.560146Eopch:     7 mean_loss = 0.724687
 900 / 900 : loss = 0.674498Eopch:     8 mean_loss = 0.699678
 900 / 900 : loss = 0.661052Eopch:     9 mean_loss = 0.691722
 900 / 900 : loss = 0.898318Eopch:    10 mean_loss = 0.676155
best_loss = 0.676155
 99 / 100  Completed buliding index in 1 seconds
topk=5, mHR=0.624800, mAP=0.545243, mRR=0.846134
topk=10, mHR=0.625800, mAP=0.522044, mRR=0.819019
topk=15, mHR=0.619600, mAP=0.505578, mRR=0.798098
topk=20, mHR=0.620800, mAP=0.499267, mRR=0.786929
Accuracy: 0.629000
[[  4   4   4  11  49]
 [  2   1   2   3  19]
 [  3   2   3   5  32]
 [  5   1   3  13  57]


In [35]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.063889, mAP=0.021430, mRR=0.250000
label=1, mHR=0.029630, mAP=0.009175, mRR=0.333333
label=2, mHR=0.046667, mAP=0.018143, mRR=1.000000
label=3, mHR=0.136709, mAP=0.055072, mRR=0.125000
label=4, mHR=0.781853, mAP=0.662917, mRR=1.000000


In [33]:
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()